In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('cleaned_items_with_metadata.csv')
queries_df = pd.read_csv('./data/queries.csv')

item_texts = data["full_text"].tolist()
query_texts = queries_df["search_term_pt"].tolist()


In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('PORTULAN/serafim-100m-portuguese-pt-sentence-encoder')

# Embed your texts
item_embeddings = model.encode(item_texts, show_progress_bar=True, batch_size=64)
query_embeddings = model.encode(query_texts, show_progress_bar=True, batch_size=64)


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
item_embeddings[0]

IndexError: invalid index to scalar variable.

In [32]:
import numpy as np

# `item_embeddings` is the result of model.encode(...)
np.save("item_embeddings_serafim.npy", item_embeddings)
np.save("query_embeddings_serafim.npy", query_embeddings)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# (100 queries) x (5000 items) similarity matrix
similarity_matrix = cosine_similarity(query_embeddings, item_embeddings)

# For each query, get Top-3 indices of most similar items
top_k = 5
top_k_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]


In [6]:
results = []

for query_idx, item_idxs in enumerate(top_k_indices):
    query_text = query_texts[query_idx]
    matched_items = [item_texts[i] for i in item_idxs]
    results.append({
        "query": query_text,
        "top_k_results": matched_items
    })


In [7]:
#based on the results get the resulting row of matched items in the original dataframe
matched_rows = []
for result in results:
    query = result["query"]
    matched_items = result["top_k_results"]
    
    for item in matched_items:
        row = data[data["full_text"] == item]
        if not row.empty:
            matched_rows.append(row.iloc[0].to_dict())

        

In [8]:
import json
test = json.loads(matched_rows[0]["itemMetadata"])

In [9]:
test

{'category_name': 'Acompanhamentos',
 'description': 'As nossas Batatas crocantes e irresistíveis, sempre levemente salgadas, servidas em nosso Balde especial! São o acompanhamento perfeito para dividir!',
 'images': ['6e73dce2-a17f-4aef-9035-1409cea198fe/202401101007_16P0_i.jpg'],
 'lacFree': False,
 'name': 'Balde de Batata Frita',
 'organic': False,
 'price': 23.99,
 'tags': [{'key': 'DIETARY_RESTRICTIONS', 'value': ['VEGETARIAN']},
  {'key': 'PORTION_SIZE', 'value': ['SERVES_2']}],
 'taxonomy': {'l0': 'ALIMENTOS_PREPARADOS',
  'l1': 'PRATOS',
  'l2': 'BATATAS_PREPARADAS'},
 'vegan': False}

In [10]:
    index = 5
    
    matched_rows = []

    query = results[index]["query"]
    matched_items = results[index]["top_k_results"]
    
    for item in matched_items:
        row = data[data["full_text"] == item]
        if not row.empty:
            matched_rows.append(row.iloc[0].to_dict())

    matched_items

["Pão de Forma 100% Nutrição Sabor Grãos Girassol E Castanha 58,6% de Cereais Integrais Wickbold  350g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}",
 "Pão de Forma 100% Nutrição Sabor Grãos Girassol E Castanha 58,6% de Cereais Integrais Wickbold  350g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}",
 "Pão de Forma Tradicional sem Casca Pullman 450g. Pacote 450g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}",
 "Pão Integral Freekeh & Nozes Wickbold 100% Nutrição Pacote 400g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}",
 "Pão Panutri 100% Integral Zero Açucar Integral 350g. Unidade 350g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}"]

In [11]:
query

'Pão rústico do campo'

In [ ]:
from openai import OpenAI

# Configure the Gemini client (OpenAI-compatible)
client = OpenAI(
    api_key="XXXXXXXX",  # Replace with your actual Gemini API key
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Reranking function
def rerank_with_gemini(query, candidates):
    candidate_text = "\n".join([f"{i+1}. {c}" for i, c in enumerate(candidates)])
    
    prompt = f"""
You are a smart food search assistant.

Given a user query and a list of candidate food items, rank them from most to least relevant to the query.

Query: "{query}"

Candidate food items:
{candidate_text}

Only return the ranking in numeric order, like this: 2, 1, 3
"""

    response = client.chat.completions.create(
        model="gemini-2.0-flash",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that ranks food items based on query relevance."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content.strip()



In [30]:
new_rank = rerank_with_gemini(query, matched_items)
print(f"Original order: {matched_items}")
print(f"New order: {new_rank}")

Original order: ["Pão de Forma 100% Nutrição Sabor Grãos Girassol E Castanha 58,6% de Cereais Integrais Wickbold  350g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}", "Pão de Forma 100% Nutrição Sabor Grãos Girassol E Castanha 58,6% de Cereais Integrais Wickbold  350g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}", "Pão de Forma Tradicional sem Casca Pullman 450g. Pacote 450g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}", "Pão Integral Freekeh & Nozes Wickbold 100% Nutrição Pacote 400g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}", "Pão Panutri 100% Integral Zero Açucar Integral 350g. Unidade 350g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}"]
New order: 4, 5, 1, 2, 3


In [31]:
reranked_items = [matched_items[int(i)-1] for i in new_rank.split(",")]
print(f"Reranked items: {reranked_items}")

Reranked items: ["Pão Integral Freekeh & Nozes Wickbold 100% Nutrição Pacote 400g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}", "Pão Panutri 100% Integral Zero Açucar Integral 350g. Unidade 350g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}", "Pão de Forma 100% Nutrição Sabor Grãos Girassol E Castanha 58,6% de Cereais Integrais Wickbold  350g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}", "Pão de Forma 100% Nutrição Sabor Grãos Girassol E Castanha 58,6% de Cereais Integrais Wickbold  350g. Pacote 400g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}", "Pão de Forma Tradicional sem Casca Pullman 450g. Pacote 450g. Categoria: Padaria. Taxonomia: {'l0': 'MERCEARIA', 'l1': 'PAES', 'l2': 'PAO_DE_FORMA'}"]
